<a href="https://colab.research.google.com/github/austin-strom/text2face/blob/main/Automatic_Image_Captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [ ]:
!gdown --id 1nD6kNAgIVjxpzIScJNLqUyRA1qEkc4Op
!gdown --id 1cwcYbl0dhXEzmdbee_K_H6jcndbsxT2o         

!unzip -u -q face2text_v0.1.zip -d face2text_v0.1
!unzip -u -q face2text_v1.0.zip -d face2text_v1.0

In [ ]:
# # This is moving the v0.1 file to the proper data dir for testing

!mkdir face2text_v0.1/data
# !mv face2text_v0.1/ Face2Text/.

!wget http://vis-www.cs.umass.edu/lfw/lfw.tgz
!tar -xvf lfw.tgz
!mv lfw face2text_v0.1/data/.

In [ ]:
!cd face2text_v0.1/data/lfw/ && ls

In [ ]:
!ls face2text_v0.1/data/lfw/Dagmar_Dunlevy

In [ ]:
!cd face2text_v0.1/ && cat clean.json

In [ ]:
import json
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text


filename = "face2text_v0.1/clean.json"
# load descriptions
doc = load_doc(filename)
with open(filename) as f:
    jj = json.load(f)
# jj = json.dumps(doc[1:-1])
print(len(jj))


In [ ]:
def load_descriptions(doc):
  mapping = dict()
  # process lines
  for line in jj:
    print(line)
    # take the first token as the image id, the rest as the description
    image_id = line["img_id"]
    image_path = line["image"]
    # convert description tokens back to string
    image_desc = line["descriptions"]
    # create the list if needed
    mapping[image_path] = list()
    # store description
    for desc in image_desc:
      print(desc)
      mapping[image_path].append(desc["text"])
  return mapping

# parse descriptions
descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))

In [ ]:
list(descriptions.keys())[:5]

In [ ]:
def clean_descriptions(descriptions):
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for key, desc_list in descriptions.items():
		for i in range(len(desc_list)):
			desc = desc_list[i]
			# tokenize
			desc = desc.split()
			# convert to lower case
			desc = [word.lower() for word in desc]
			# remove punctuation from each token
			desc = [w.translate(table) for w in desc]
			# remove hanging 's' and 'a'
			desc = [word for word in desc if len(word)>1]
			# remove tokens with numbers in them
			desc = [word for word in desc if word.isalpha()]
			# store as string
			desc_list[i] =  ' '.join(desc)

# clean descriptions
clean_descriptions(descriptions)

In [ ]:
# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
	# build a list of all description strings
	all_desc = set()
	for key in descriptions.keys():
		[all_desc.update(d.split()) for d in descriptions[key]]
	return all_desc

# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Original Vocabulary Size: %d' % len(vocabulary))

In [ ]:
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
  lines = list()
  for key, desc_list in descriptions.items():
    for desc in desc_list:
      lines.append(str(key) + ' ' + desc)
  data = '\n'.join(lines)
  file = open(filename, 'w')
  file.write(data)
  file.close()

save_descriptions(descriptions, 'descriptions.txt')

In [ ]:
!sed -i 's/bmp/jpg/g' descriptions.txt

In [ ]:
# load a pre-defined list of photo identifiers
def load_set(filename):
  name_mapping = []
  # process lines
  for line in jj:
    # take the first token as the image id, the rest as the description
    # name_mapping.append( line["img_id"])
    name_mapping.append( line["image"])
  return name_mapping

# load training dataset (6K)
train = load_set(filename)
print('Dataset: %d' % len(train))

In [ ]:
# Below path contains all the images
images = 'face2text_v0.1/data/lfw/'
# Create a list of all image names in the directory
img = glob.glob(images + '**/*.jpg')
print(len(img))
# !ls face2text_v0.1/data/lfw


In [ ]:
# # Below file conatains the names of images to be used in train data
# train_images_file = "face2text_v0.1/clean.json"
# # Read the train image names in a set
# train_images = set(open(train_images_file, 'r').read().strip().split('\n'))

# # Create a list of all the training images with their full path names
# train_img = []

# for i in img: # img is list of full path names of all images
#     if i[len(images):] in train_images: # Check if the image belongs to training set
#         train_img.append(i) # Add it to the list of train images

### Use ImageMagik??

In [ ]:
def load_set_name(filename):
  name_mapping = []
  # process lines
  for line in jj:
    # take the first token as the image id, the rest as the description
    name_mapping.append( line["image"].replace('bmp', 'jpg'))

  return name_mapping
# print(len(image_names))
print(len(descriptions))
image_names = load_set_name(filename)
train_img = image_names[:300]
test_img = image_names[300:]

In [ ]:
print(image_names)

In [ ]:
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
  # load document
  doc = load_doc(filename)
  descriptions = dict()
  for line in doc.split('\n'):
    # print(line)
    # split line by white space
    tokens = line.split()
    # split id from description
    image_id, image_desc = tokens[0], tokens[1:]
    # skip images not in the set
    print(tokens)
    if image_id in dataset:
      # create list
      if image_id not in descriptions:
        descriptions[image_id] = list()
      # wrap description in tokens
      desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
      # store
      descriptions[image_id].append(desc)
  return descriptions

# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', image_names)
print('Descriptions: train=%d' % len(train_descriptions))

In [ ]:
def preprocess(image_path):
    # Convert all the images to size 299x299 as expected by the inception v3 model
    img = image.load_img(image_path, target_size=(299, 299))
    # Convert PIL image to numpy array of 3-dimensions
    x = image.img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    return x

In [ ]:
# Load the inception v3 model
model = InceptionV3(weights='imagenet')

In [ ]:
# Create a new model, by removing the last layer (output layer) from the inception v3
model_new = Model(model.input, model.layers[-2].output)

In [ ]:
# Function to encode a given image into a vector of size (2048, )
def encode(image):
    image = preprocess(image) # preprocess the image
    fea_vec = model_new.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec

In [ ]:
print(train_img)

In [ ]:
print(images)

In [ ]:
# Call the funtion to encode all the train images
# This will take a while on CPU - Execute this only once
start = time()
encoding_train = {}

for img in train_img:
  encoding_train[img] = encode(images + img)
print("Time taken in seconds =", time()-start)

In [ ]:
# Save the bottleneck train features to disk
import pickle
with open(filename + "encoded_train_images.pkl", "wb") as encoded_pickle:
    pickle.dump(encoding_train, encoded_pickle)

In [ ]:
# Call the funtion to encode all the test images - Execute this only once
start = time()
encoding_test = {}
for img in test_img:

    # encoding_test[img[len(images):]] = encode(images+img)
    encoding_test[img] = encode(images+img)

print("Time taken in seconds =", time()-start)

In [ ]:
# Save the bottleneck test features to disk
with open( filename + "encoded_test_images.pkl", "wb") as encoded_pickle:
    pickle.dump(encoding_test, encoded_pickle)

In [ ]:
train_features = load(open(filename + "encoded_train_images.pkl", "rb"))
print('Photos: train=%d' % len(train_features))

In [ ]:
print(encoding_test['Dagmar_Dunlevy/Dagmar_Dunlevy_0001.jpg'])
print(len(train_descriptions))

In [ ]:
# Create a list of all the training captions
all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)
len(all_train_captions)

In [ ]:
# Consider only words which occur at least 10 times in the corpus
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
print('preprocessed words %d -> %d' % (len(word_counts), len(vocab)))

In [ ]:
ixtoword = {}
wordtoix = {}

ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

In [ ]:
vocab_size = len(ixtoword) + 1 # one for appended 0's
vocab_size

In [ ]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip


In [ ]:
!ls

# Load Glove vectors
embeddings_index = {} # empty dictionary
f = open(os.path.join('glove.6B.200d.txt'), encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
embedding_dim = 200

# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape
print(embedding_matrix)

In [ ]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            # TODO: Key is currently the ID, which is just the number. Might change to the list index
    
            photo = photos[key]
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)

            # yield the batch data
            if n==num_photos_per_batch:
                # yield [[array(X1), array(X2)], array(y)]
                yield ([array(X1), array(X2)], array(y))
                X1, X2, y = list(), list(), list()
                n=0

In [ ]:
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [ ]:
# model.summary()

In [ ]:
model.layers[2]

In [ ]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
epochs = 10
number_pics_per_bath = 3
steps = len(train_descriptions)//number_pics_per_bath

In [ ]:
# generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
# print(next(generator))
# train_features['Dagmar_Dunlevy/Dagmar_Dunlevy_0001.jpg']

In [ ]:
keys = list(train_descriptions.keys())
print(keys)
train_keys = keys[:300]
test_keys = keys[300:]

train_subset = {key: value for key, value in train_descriptions.items() if key in train_keys}
print(len(train_subset))
test_subset = {key: value for key, value in train_descriptions.items() if key in test_keys}
print(len(test_subset))
for i in range(epochs):
    generator = data_generator(train_subset, train_features, wordtoix, max_length, number_pics_per_bath)
    # model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)

    model.save('./model_weights/model_' + str(i) + '.h5')

In [ ]:
model.optimizer.lr = 0.0001
epochs = 10
number_pics_per_bath = 6
steps = len(train_descriptions)//number_pics_per_bath

In [ ]:
model.save_weights('./model_weights/model_30.h5')

In [ ]:
model.load_weights('./model_weights/model_30.h5')

In [ ]:
# images = '../../storage/image_caption/dataset/Flicker8k_Dataset/'

In [ ]:
# with open("../../storage/image_caption/dataset/Pickle/encoded_test_images.pkl", "rb") as encoded_pickle:
#     encoding_test = load(encoded_pickle)

In [ ]:
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final
z=0

In [ ]:
z+=1
pic = list(encoding_test.keys())[z]
image = encoding_test[pic].reshape((1,2048))
x=plt.imread(images+pic)
plt.imshow(x)
plt.show()
print("Greedy:",greedySearch(image))